In [ ]:
# this file is for temporarily matching figuring out how to match what we have in the model to the lit data 
# easier to debugg in here!


In [9]:
#import polars as pl
import numpy as np
from matplotlib import pyplot as plt
import os
import json
import pandas as pd
#import seaborn as sns
from scipy.sparse import csr_matrix

In [25]:
# Load in the data sources that we currently use: 
# TODO: not sure why, but I cannot figure out how to import these not using the absolute path...

Nend_data = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/protein_half_lives_n_end_rule.tsv", comment="#")
exp_data = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/protein_half_lives_measured.tsv", comment="#")
ML_data = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/protein_half_lives_pulsed_silac.tsv", comment="#")

In [32]:
# Load in the experimental data from the new paper (Gupta et al., 2023), Table S1 (https://www.biorxiv.org/content/10.1101/2022.08.01.502339v2.full): 

# Measured half lives (from Gupta et al., 2023 bioRxiv)
new_data = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/protein_half_lives_gupta.tsv", comment="#")
#TODO: figure out to read out the new data with out having to use my aboslute file path. NOt sure why it isnt working....

## figure out what all the current monomers in the model are:

In [29]:
# N-end rule calculated protein degredation rates

# Half lives of protein monomers are determined by the identity of their amino-terminal residues. Slow degradation is assumed for P and U. (Tobias et al, 1986)
Nend_monomers = Nend_data.aa_id
# TODO: this is the only one I am sort of not sure how stuff in it is defined? @Nora do you know what these are and how they could relate to protein/monomer names?
Nend_monomers

0      L-ALPHA-ALANINE[c]
1                  ARG[c]
2                  ASN[c]
3          L-ASPARTATE[c]
4                  CYS[c]
5                  GLT[c]
6                  GLN[c]
7                  GLY[c]
8                  HIS[c]
9                  ILE[c]
10                 LEU[c]
11                 LYS[c]
12                 MET[c]
13                 PHE[c]
14                 PRO[c]
15                 SER[c]
16                 THR[c]
17                 TRP[c]
18                 TYR[c]
19    L-SELENOCYSTEINE[c]
20                 VAL[c]
Name: aa_id, dtype: object

In [30]:
# Experimentally measured protein degredation rates 

# Measured half lives (from Macklin et al., 2020)
exp_monomers = exp_data.id
exp_monomers

0                DCUR-MONOMER
1    DETHIOBIOTIN-SYN-MONOMER
2             EG10863-MONOMER
3              CARBPSYN-SMALL
4             EG10743-MONOMER
5          GLUTCYSLIG-MONOMER
6         CDPDIGLYSYN-MONOMER
Name: id, dtype: object

In [31]:
# ML DATA

# Generated by reconstruction/ecoli/scripts/protein_half_lives/convert_to_flat.py on Mon May  9 13:39:16 2022 
# using the methods in https://pubmed.ncbi.nlm.nih.gov/33531410/
ML_monomers = ML_data.id
ML_monomers

0                   2-ISOPROPYLMALATESYN-MONOMER
1       2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER
2       3-CH3-2-OXOBUTANOATE-OH-CH3-XFER-MONOMER
3                 3-ISOPROPYLMALDEHYDROG-MONOMER
4                   3-OXOACYL-ACP-REDUCT-MONOMER
                          ...                   
1088                                        YIDC
1089                                YJCX-MONOMER
1090                                YJJK-MONOMER
1091                                YLIB-MONOMER
1092                                ZNUA-MONOMER
Name: id, Length: 1093, dtype: object

In [102]:
# New Data
new_monomers = new_data["Protein name"]
new_genes = new_data["Gene name"]

## Map the monomer names in the current data to gene names:

In [66]:
# Obtain the gene id and monomer id mappings: 

rnas = pd.read_table("/Users/miagrahn/wcEcoli/reconstruction/ecoli/flat/rnas.tsv", comment="#")

m_ids = rnas.monomer_ids
monomer_ids = []
for i in range(len(m_ids)):
	m = m_ids[i]
	m = m.replace("[",""); m = m.replace("]","")
	m = m.replace('"','')
	monomer_ids.append(m)
	
gene_ids = rnas.gene_id
gene_CNs = rnas.common_name

# make organized libraries of monomer names to indexes: 
monomer_idx_dict = {id: i for i, id in enumerate(monomer_ids)}
idx_monomer_dict = {idx: i for i, idx in monomer_idx_dict.items()}

gene_idx_dict = {id: i for i, id in enumerate(gene_ids)}
idx_gene_dict = {idx: i for i, idx in gene_idx_dict.items()}

GCN_idx_dict = {id: i for i, id in enumerate(gene_CNs)}
idx_GCN_dict = {idx: i for i, idx in GCN_idx_dict.items()}


In [ ]:
# Figure out which gene each monomer in the N end rule data set corresponds to: 

# TODO: figure out the amino acid part of this
# TODO: i think that these ones might be included in the list for the exp data anyway, double check with nora what she thinks by just showing her what the exp_data looks lik e

In [101]:
# Figure out which gene each monomer in the experimental data set corresponds to: 
exp_match_Pids = []
exp_match_idxs = [] 
exp_match_Gids = []
exp_match_GCNids = []

too_many_exp_match_Pidxs = []
non_exp_match_Pids = []
non_exp_match_Pidxs = []

for i in range(len(exp_monomers)):
	m = str(exp_monomers[i])
	index = [name for key, name in monomer_idx_dict.items() if m in key]
	if len(index) > 1: 
		print("too many?")
		too_many_exp_match_Pidxs.append(index)
	if len(index) == 0:
		index = index[0]
		non_exp_match_Pids.append(m)
		non_exp_match_Pidxs.append(i)
	else: 
		index = index[0]
		exp_match_Pids.append(m)
		exp_match_idxs.append(i)
		gene_id = idx_gene_dict.get(index)
		exp_match_Gids.append(gene_id)
		gene_CN = idx_GCN_dict.get(index)
		exp_match_GCNids.append(gene_CN)
		
print(len(exp_match_Pids))
exp_match_Pids

7


['DCUR-MONOMER',
 'DETHIOBIOTIN-SYN-MONOMER',
 'EG10863-MONOMER',
 'CARBPSYN-SMALL',
 'EG10743-MONOMER',
 'GLUTCYSLIG-MONOMER',
 'CDPDIGLYSYN-MONOMER']

In [98]:
# Figure out which gene each monomer in the ML data set corresponds to: 
ML_match_Pids = []
ML_match_idxs = [] 
ML_match_Gids = []
ML_match_GCNids = []

too_many_ML_match_Pids = []
too_many_ML_match_Pidxs = []
non_ML_match_Pids = []
non_ML_match_Pidxs = []

for i in range(len(ML_monomers)):
	m = str(ML_monomers[i])
	#TODO: figure out if we want exact == here or "in" here (it changes the result) 
	index = [name for key, name in monomer_idx_dict.items() if m == key]
	if len(index) > 1: 
		print("too many?")
		print(index)
		print(m)
		too_many_ML_match_Pids.append(m)
		too_many_ML_match_Pidxs.append(index)
	if len(index) == 0:
		non_ML_match_Pids.append(m)
		non_ML_match_Pidxs.append(i)
	else: 
		index = index[0]
		ML_match_Pids.append(m)
		ML_match_idxs.append(i)
		gene_id = idx_gene_dict.get(index)
		ML_match_Gids.append(gene_id)
		gene_CN = idx_GCN_dict.get(index)
		ML_match_GCNids.append(gene_CN)
		
print(len(ML_match_Pids))
print(len(too_many_ML_match_Pidxs))
print(len(non_ML_match_Pidxs))

1087
0
6


In [103]:
len(monomer_ids)
# TODO: why do none of the current data add up to this 4748 number? shouldnt they?

4748

In [104]:
len(ML_monomers)

1093

In [109]:
len(Nend_monomers)

21

In [115]:
exp_data

,id,half life (units.min),_comments
0,DCUR-MONOMER,108,DcuR
1,DETHIOBIOTIN-SYN-MONOMER,324,BioD
2,EG10863-MONOMER,36,Rph
3,CARBPSYN-SMALL,846,CarA
4,EG10743-MONOMER,972,Pnp
5,GLUTCYSLIG-MONOMER,2382,GshA
6,CDPDIGLYSYN-MONOMER,600,"CdsA, unable to measure exactly but measured a..."


In [116]:
# complie a list of ALL the current commen name gene ids from the current model data:
old_GCNs = exp_match_GCNids + ML_match_GCNids
old_genes = exp_match_Gids + ML_match_Gids
old_monomers = exp_match_Pids + ML_match_Pids

# find the protein degradation rate constant for each known set of indices:
exp_match_PDRs = []
for i in exp_match_idxs:
	PDR = exp_data["half life (units.min)"][i]
	exp_match_PDRs.append(PDR)
	
ML_match_PDRs = []
for i in ML_match_idxs:
	PDR = ML_data["half_life (units.min)"][i]
	ML_match_PDRs.append(PDR)
	
# put all the protein deg rates together: 
old_PDRs = exp_match_PDRs + ML_match_PDRs

old_data = {'GCN': old_GCNs, 'gene id': old_genes, 'monomer id': old_monomers, 'PDR': old_PDRs}
old_df = pd.DataFrame(old_data)
old_df

,GCN,gene id,monomer id,PDR
0,dcuR,G7826,DCUR-MONOMER,108.0
1,bioD,EG10120,DETHIOBIOTIN-SYN-MONOMER,324.0
2,rph,EG10863,EG10863-MONOMER,36.0
3,carA,EG10134,CARBPSYN-SMALL,846.0
4,pnp,EG10743,EG10743-MONOMER,972.0
...,...,...,...,...
1089,yidC,EG11197,YIDC,241.4
1090,alsB,EG12458,YJCX-MONOMER,51.3
1091,ettA,EG12343,YJJK-MONOMER,2829.2
1092,gsiB,G6430,YLIB-MONOMER,254.1


## Map the new data to the monomers in the model:

In [106]:
# Figure out which gene each monomer in the new data set corresponds to: 
# TODO: so note that none of the new data monomers match the names in the original monomer list, so maybe compare the gene id directly since I have it 
new_match_Pids = []
new_match_idxs = [] 
new_match_Gids = []
new_match_GCNids = []

too_many_new_match_Pids = []
too_many_new_match_Pidxs = []
non_new_match_Pids = []
non_new_match_Pidxs = []

for i in range(len(new_monomers)):
	m = str(new_monomers[i])
	index = [name for key, name in monomer_idx_dict.items() if m == key]
	if len(index) > 1: 
		print("too many?")
		too_many_new_match_Pids.append(m)
		too_many_new_match_Pidxs.append(index)
	if len(index) == 0:
		non_new_match_Pids.append(m)
		non_new_match_Pidxs.append(i)
	else: 
		index = index[0]
		new_match_Pids.append(m)
		new_match_idxs.append(i)
		gene_id = idx_gene_dict.get(index)
		new_match_Gids.append(gene_id)
		gene_CN = idx_GCN_dict.get(index)
		new_match_GCNids.append(gene_CN)
		
print(len(new_match_Pids))
print(len(too_many_new_match_Pidxs))
print(len(non_new_match_Pidxs))

0
0
3253


In [111]:
# Map new data gene common names matched to those in the current model entire list: 
# TODO: ask nora what to do about the fact that some of the protiens/genes in the new data that do not have a rate constant reported for them ( like towards the bottom of the excel, you can see not all protiens have a rate reported for them. should we be filtering those out first somehow?) RN i think its sort of doing all of them, I need to change that (i can only see the first 100 in the short preview too)
new_match_Pids = []
new_match_idxs = [] 
new_match_Gids = []
new_match_GCNids = []

too_many_new_match_GCNids = []
too_many_new_match_GCNidxs = []
non_new_match_GCNids = []
non_new_match_GCNidxs = []

for i in range(len(new_genes)):
	m = str(new_genes[i])
	index = [name for key, name in GCN_idx_dict.items() if m == key]
	if len(index) > 1: 
		print("too many?")
		too_many_new_match_GCNids.append(m)
		too_many_new_match_GCNidxs.append(index)
	if len(index) == 0:
		non_new_match_GCNids.append(m)
		non_new_match_GCNidxs.append(i)
	else: 
		index = index[0]
		new_match_GCNids.append(m)
		new_match_idxs.append(i)
		gene_id = idx_gene_dict.get(index)
		new_match_Gids.append(gene_id)
		monomer_id = idx_monomer_dict.get(index)
		new_match_Pids.append(monomer_id)
		
print(len(new_match_Pids))
print(len(too_many_new_match_Pidxs))
print(len(non_new_match_Pidxs))

3089
0
3253


In [117]:
new_data

,Unnamed: 0,Uniprot accession,Protein name,Gene name,MinimalMedia_1,MinimalMedia_2,MinimalMedia_mean
0,1,A5A614,YCIZ_ECOLI,yciZ,0.442631,NaN,0.442631
1,2,O32583,THIS_ECOLI,thiS,0.424812,0.673641,0.549227
2,3,P00350,6PGD_ECOLI,gnd,0.902359,0.798815,0.850587
3,4,P00363,FRDA_ECOLI,frdA,0.768480,0.780088,0.774284
4,5,P00370,DHE4_ECOLI,gdhA,0.759368,0.670745,0.715057
...,...,...,...,...,...,...,...
3248,3249,P77736,YAHD_ECOLI,yahD,NaN,NaN,NaN
3249,3250,Q2M7X4,YICS_ECOLI,yicS,NaN,NaN,NaN
3250,3251,Q46796,YGEP_ECOLI,ygeP,NaN,NaN,NaN
3251,3252,Q47013,ELAD_ECOLI,elaD,NaN,NaN,NaN


In [118]:
# complie a list of ALL the new commen name gene ids from the current model data:
# TODO: ask nora waht the units are on these PDRs 

# find the protein degradation rate constant for each known set of indices:
new_match_PDRs = []
for i in new_match_idxs:
	PDR = new_data["MinimalMedia_mean"][i]
	new_match_PDRs.append(PDR)

new_df_data = {'GCN': new_match_GCNids, 'gene id': new_match_Gids, 'monomer id': new_match_Pids, 'PDR': new_match_PDRs}
new_df = pd.DataFrame(new_df_data)
new_df

,GCN,gene id,monomer id,PDR
0,yciZ,G0-10591,MONOMER0-2820,0.442631
1,thiS,G1,THIS-MONOMER,0.549227
2,gnd,EG10411,6PGLUCONDEHYDROG-MONOMER,0.850587
3,frdA,EG10330,FUM-FLAVO,0.774284
4,gdhA,EG10372,GDHA-MONOMER,0.715057
...,...,...,...,...
3084,yahD,G6183,G6183-MONOMER,NaN
3085,yicS,G0-10469,MONOMER0-2691,NaN
3086,ygeP,G7482,G7482-MONOMER,NaN
3087,elaD,G7176,G7176-MONOMER,NaN


In [125]:
# match the new and old data together now! 
matches = []
match_idxs = []
match_Pids = []
match_Gids = [] 
match_GCNs = []
match_old_PDRs = []
match_new_PDRs = [] 

no_match_new_to_old_GCNs = []
no_match_new_to_old_idxs = []

old_GCN_idx_dict = {id: i for i, id in enumerate(old_GCNs)}
old_idx_GCN_dict = {idx: i for i, idx in old_GCN_idx_dict.items()}

for i in range(len(new_match_GCNids)):
	GCN = new_match_GCNids[i]
	index = [name for key, name in old_GCN_idx_dict.items() if GCN == key]
	if len(index) == 0:
		no_match_new_to_old_GCNs.append(GCN)
		no_match_new_to_old_idxs.append(i)
	else: 
		index = index[0]
		match_GCNs.append(GCN)
		match_idxs.append(i)
		gene_id = old_genes[index]
		match_Gids.append(gene_id)
		p_id = old_monomers[index]
		match_Pids.append(p_id)
		old_PDR = old_PDRs[index]
		match_old_PDRs.append(old_PDR)
		new_PDR = new_match_PDRs[i]
		match_new_PDRs.append(new_PDR)
	
# make a total data frame: 
match_df = {'GCN': match_GCNs, 'gene id': match_Gids, 'monomer id': match_Pids, 'new PDR': match_new_PDRs, 'old PDR': match_old_PDRs}
match_df = pd.DataFrame(match_df)
match_df

,GCN,gene id,monomer id,new PDR,old PDR
0,gnd,EG10411,6PGLUCONDEHYDROG-MONOMER,0.850587,390.7
1,gdhA,EG10372,GDHA-MONOMER,0.715057,484.0
2,ndh,EG10649,NADH-DHII-MONOMER,1.009853,316.0
3,sodA,EG10953,SUPEROX-DISMUTMN-MONOMER,0.806555,128.3
4,nrdA,EG10660,NRDA-MONOMER,0.858176,266.5
...,...,...,...,...,...
1073,glf,EG11981,GALPMUT-MONOMER,NaN,159.6
1074,yjdJ,G7829,G7829-MONOMER,NaN,52.5
1075,ynfM,G6854,B1596-MONOMER,NaN,5000.0
1076,csgG,G6543,G6543-MONOMER,NaN,114.3


In [133]:
# how many in the new data set were not found in the old dataset?
print(f"{len(no_match_new_to_old_GCNs)} monomers in the new dataset were not found in the old dataset")

2011 monomers in the new dataset were not found in the old dataset


In [127]:
# have the reverse method coded so that I can figure out which of the new monomers are not in the old set: 
match_idxs = []
match_Pids = []
match_Gids = [] 
match_GCNs = []
match_old_PDRs = []
match_new_PDRs = [] 

# these will help figure out which are missing and need to be replaced
no_match_old_to_new_GCNs = []
no_match_old_to_new_idxs = []

new_GCN_idx_dict = {id: i for i, id in enumerate(new_match_GCNids)}
new_idx_GCN_dict = {idx: i for i, idx in new_GCN_idx_dict.items()}

for i in range(len(old_GCNs)):
	GCN = old_GCNs[i]
	index = [name for key, name in new_GCN_idx_dict.items() if GCN == key]
	if len(index) == 0:
		no_match_old_to_new_GCNs.append(GCN)
		no_match_old_to_new_idxs.append(i)
		match_GCNs.append(GCN)
		gene_id = old_genes[i]
		match_Gids.append(gene_id)
		p_id = old_monomers[i]
		match_Pids.append(p_id)
		old_PDR = old_PDRs[i]
		match_old_PDRs.append(old_PDR)
		#TODO: ask Nora if she thinks I should just make this value that of the old data set, so that we can use one data colum easily going forward
		new_PDR = "NA"
		match_new_PDRs.append(new_PDR)
	else: 
		index = index[0]
		match_GCNs.append(GCN)
		match_idxs.append(i)
		gene_id = new_match_Gids[index]
		match_Gids.append(gene_id)
		p_id = new_match_Pids[index]
		match_Pids.append(p_id)
		old_PDR = old_PDRs[i]
		match_old_PDRs.append(old_PDR)
		new_PDR = new_match_PDRs[index]
		match_new_PDRs.append(new_PDR)
	
# make a total data frame: 
old_to_new_match_df = {'GCN': match_GCNs, 'gene id': match_Gids, 'monomer id': match_Pids, 'new PDR': match_new_PDRs, 'old PDR': match_old_PDRs}
old_to_new_match_df = pd.DataFrame(old_to_new_match_df)
old_to_new_match_df

,GCN,gene id,monomer id,new PDR,old PDR
0,dcuR,G7826,DCUR-MONOMER,0.557266,108.0
1,bioD,EG10120,DETHIOBIOTIN-SYN-MONOMER,0.764226,324.0
2,rph,EG10863,EG10863-MONOMER,0.891793,36.0
3,carA,EG10134,CARBPSYN-SMALL,0.741366,846.0
4,pnp,EG10743,EG10743-MONOMER,0.760874,972.0
...,...,...,...,...,...
1089,yidC,EG11197,YIDC,0.674876,241.4
1090,alsB,EG12458,YJCX-MONOMER,0.823808,51.3
1091,ettA,EG12343,YJJK-MONOMER,0.751185,2829.2
1092,gsiB,G6430,YLIB-MONOMER,0.735787,254.1


In [131]:
# see how many monomers from the old dataset were not found in the new dataset: 
print(f"{len(no_match_old_to_new_GCNs)} monomers in the old dataset were not found in the new dataset")
no_match_old_to_new_GCNs

15 monomers in the old dataset were not found in the new dataset


['cdsA',
 'argF',
 'rfbD',
 'rpmG',
 'hdeA',
 'dicD',
 'yacG',
 'ydiE',
 'evgA',
 'ydbL',
 'pqqU',
 'galF',
 'yffS',
 'gatA',
 'osmF']